In [2]:
# check getloader return type supposed to be lr_image, hr_image


In [1]:
# check model architecuture, whethere there are incopatible channel sizes
from models import LitSRResnet, SRResnetModelv1
config = SRResnetConfig(
    out_channels=1,
    in_channels=1,
    hidden_channel=2,
    lr=1e-3,
    num_blocks=2,
    is_training=True
)
model = LitSRResnet(config)
print(model.config)
print("-"*50)
print(model)
print("-"*50)
x = torch.randn(1,1,64,64)
y = model(x)
print("\nresults = ", y.shape, f"resolution upscale size= {y.size(2)/x.size(2)}x")
_model = SRResnetModelv1(config)
print(_model)
print("-"*50)
y = _model(x)
print("\nresults = ", y.shape, f"resolution upscale size= {y.size(2)/x.size(2)}x")



In [3]:
# train using lightning